## Build a Solar Atmosphere Using McWhirter and VALIIc Data

### Steps
1. Set parameters for model e.g. physical dimensions and grid size
2. Load VALIIIc and McWhirter Data
3. Interpolate Data
4. Regenerat model


In [2]:
import os
import numpy as np

import astropy.table
from astropy.table import Table
import astropy.units as u
from astropy.constants import k_B, m_p
import scipy.constants as asc
from astropy.table import Table


from scipy.interpolate import UnivariateSpline

from sacio import *
#alldat,modelinfo=read_sac_ascii('../../../configs/hydro/3D_128_spic_asc.ini')

In [ ]:
alldat,modelinfo=read_sac_ascii('../../solar-testdata/3D_128_spic_bvertbg100G_asc.ini')



read in binary output 
rewrite as ascii
re-readascii file

In [ ]:
print(modelinfo)
print(np.size(alldat))
print(np.shape(alldat))

#Display x co-ordinates here vertical section through atmosphere
print(alldat[48:80,64,64,0]/1.0e3)

In [3]:
alldat1,modelinfo1=read_sac_bin('../../solar-testdata/zerospic1__400000.out')

sacio.py:197: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  dim = np.fromfile(file,dtype=np.int32,count=ndim)[:ndim]
sacio.py:212: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  alldat=np.fromfile(file,dtype=float,count=(nfields+ndim)*dim[0]*dim[1]*dim[2])[:(nfields+ndim)*dim[0]*dim[1]*dim[2]]
/home/mike/.conda/envs/pysac/lib/python2.7/site-packages/numpy/core/fromnumeric.py:224: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  return reshape(newshape, order=order)


In [4]:
print(modelinfo1[6])
print(np.shape(alldat1))
print(alldat1[3,48:80,64,64]/1.0e3)

[128 128 128]
(16, 128, 128, 128)
[ -6.47514769e-18  -8.06240523e-18  -9.29623523e-18  -1.04858154e-17
  -1.16886669e-17  -1.26721044e-17  -1.36518754e-17  -1.45404664e-17
  -1.52007798e-17  -1.56662482e-17  -1.59772105e-17  -1.62569796e-17
  -1.65168071e-17  -1.66981893e-17  -1.68764172e-17  -1.70454817e-17
  -1.71881961e-17  -1.73124828e-17  -1.74050967e-17  -1.74781906e-17
  -1.75382649e-17  -1.75669360e-17  -1.75521429e-17  -1.73886237e-17
  -1.70206534e-17  -1.65092876e-17  -1.59809209e-17  -1.55017893e-17
  -1.48685448e-17  -1.40471181e-17  -1.32185854e-17  -1.23207119e-17]


## Test the binary sac output writer

In [ ]:
write_sac_bin('test.out',alldat1,modelinfo1)

(array([128, 128, 128], dtype=int32), array([3], dtype=int32), array([13], dtype=int32), array([7], dtype=int32))


### Extracting some of the data fields

In [ ]:
dens=alldat[:,:,:,3]

#X=alldat[0,64,:,:]/1.0e6
#Y=alldat[1,64,:,:]/1.0e6
#vx=alldat[4,64,:,:]/(alldat[3,64,:,:]+alldat[12,64,:,:])
#vy=100000*alldat[5,64,:,:]/(alldat[3,64,:,:]+alldat[12,64,:,:])
#vz=100000*alldat[6,64,:,:]/(alldat[3,64,:,:]+alldat[12,64,:,:])
#rho=(alldat[3,64,:,:])*1.0e12
#e=alldat[7,64,:,:]
#vx=alldat[4,64,:,:]
#dens=(alldat[3,64,:,:]+alldat[12,64,:,:])
#bx=alldat[8,64,:,:]*sqrt(mu)*1.0e4
#by=alldat[9,64,:,:]*sqrt(mu)*1.0e4
#bz=alldat[10,64,:,:]*sqrt(mu)*1.0e4
print(alldat[48:80,64,64,12])

In [ ]:
from valmcw import *
data=val3c()
#data.sort('Z')
print(data)

In [ ]:
__all__ = ['read_VAL3c_MTW', 'interpolate_atmosphere', 'get_spruit_hs', 'vertical_profile']

In [ ]:
#if 1D or 2D set unused dimensions to 0, and unrequired xyz limits to 1. 
Nxyz = [128,128,128] # 3D grid
xyz_SI = [-1e6,1e6,-1e6,1e6,3.5e4,1.6e6] # xyz limits SI/CGS units 
xyz  = [-1.27*u.Mm,1.27*u.Mm,-1.27*u.Mm,1.27*u.Mm,0.*u.Mm,8.62*u.Mm] #grid size
#xyz.unit=u.Mm
#xyz=np.array([-1e6,1e6,-1e6,1e6,3.5e4,1.6e6])
#xyz = [-1*u.Mm,1e6,-1*u.Mm,1*u.Mm,0.035*u.Mm,1.6*u.Mm] # xyz limits SI/CGS units  
#xyz=xyz_SI

In [ ]:
"""
get_coords returns a non-dimensional dictionary describing the domain
coordinates.
"""
dz=(xyz[5]-xyz[4])/(Nxyz[2]-1)
#Z    = u.Quantity(np.linspace(xyz[4].value, xyz[5].value, Nxyz[2]), unit=xyz.unit)
Z    = u.Quantity(np.linspace(xyz[4].value, xyz[5].value, Nxyz[2]))
#Zext = u.Quantity(np.linspace(Z.min().value-4.*dz.value, Z.max().value+4.*dz.value, Nxyz[2]+8), unit=Z.unit)
Zext = u.Quantity(np.linspace(Z.min().value-4.*dz.value, Z.max().value+4.*dz.value, Nxyz[2]+8))
coords = {
              'dx':(xyz[1]-xyz[0])/(Nxyz[0]-1),
              'dy':(xyz[3]-xyz[2])/(Nxyz[1]-1),
              'dz':(xyz[5]-xyz[4])/(Nxyz[2]-1),
              'xmin':xyz[0],
              'xmax':xyz[1],
              'ymin':xyz[2],
              'ymax':xyz[3],
              'zmin':xyz[4],
              'zmax':xyz[5],
              'Z':Z,
              'Zext':Zext
            }

In [ ]:
#identify location of source data files
__files__=''
homedir = os.environ['HOME']
cwd = os.path.dirname(__files__)
#cwd = homedir+'/Dropbox/multi_tube/python/allpapers/'
VAL_file = os.path.join(cwd, 'VALIIIC.dat')
MTW_file = os.path.join(cwd, 'mcwhirter.dat')

filenames = [VAL_file, MTW_file]
# uncomment and switch to l_const/l_sqrt/l_linear/l_square as required  
#logical_pars['l_square'] = True 

In [ ]:
#============================================================================
    # Dimensional units in terms of SI
#============================================================================
scales   = {
            'length':         1e2*u.Mm,
            'density':        1e-4*u.kg/u.m**3,
            'velocity':       1e2*u.m/u.s,
            'temperature':    1.0*u.K, 
            'magnetic':       1e-3*u.T #mT
           }


scales['energy density'] = scales['density'] * scales['velocity']**2
scales['time'] = scales['length'] / scales['velocity'] 
scales['mass'] = scales['density'] * scales['length']**3 
scales['force density'] = scales['density'] * scales['velocity'] / \
scales['time'] #D momentum/Dt force density balance 

In [ ]:
print(5*scales['temperature'])

print(coords)

In [ ]:
print (xyz[1]-xyz[0])/(Nxyz[0]-1)

In [ ]:
from scipy.interpolate import UnivariateSpline
s=0.25
hdata = np.array(u.Quantity(data['Z']).to(u.m))
# interpolate total pressure, temperature and density profiles
pdata_f = UnivariateSpline(hdata,np.array(np.log(data['p'])),k=1, s=s)
Tdata_f = UnivariateSpline(hdata,np.array(np.log(data['T'])),k=1, s=s)
rdata_f = UnivariateSpline(hdata,np.array(np.log(data['rho'])),k=1, s=s)
#s=0.0 to ensure all points are strictly used for ionisation state
#muofT_f = UnivariateSpline(hdata,np.array(np.log(data['mu'])),k=1, s=0.0)
#print pdata_f(Z)
#print Z
print scales['temperature']
print(Tdata_f(hdata))
#print((data['Z']))
print Nxyz[0]
outdata = Table([np.zeros((Nxyz[0])),np.zeros((Nxyz[0])),np.zeros((Nxyz[0])),np.zeros((Nxyz[0]))],names=['Z', 'T','p','rho'])
outdata['Z'] = Z
#outdata['p'] = np.exp(pdata_f(Z.to(u.m))) 
#outdata['p'] = np.exp(pdata_f(Z.to(u.cm))) * data['p'].unit
outdata['p'] = np.exp(pdata_f(Z*1.0e8)) * data['p'].unit
#outdata['T'] = np.exp(Tdata_f(Z.to(u.m))) * data['T'].unit
#outdata['T'] = np.exp(Tdata_f()) * data['T'].unit 
outdata['T'] = np.exp(Tdata_f(Z*1.0e6)) * data['T'].unit
#outdata['rho'] = np.exp(rdata_f(Z.to(u.m))) * data['rho'].unit
#outdata['rho'] = np.exp(rdata_f()) * data['rho'].unit 
outdata['rho'] = np.exp(rdata_f(Z*1.0e8)) * data['rho'].unit 
#outdata['mu'] = np.exp(muofT_f(Z.to(u.m))) * u.one



In [ ]:
print outdata['rho'][1]

#VAL3c = np.loadtxt(filenames[0])[::-1,:]
#VAL3c[:,0] *= 1e3 #km -> m
#VAL3c[:,1] *= 1e3 #g/cm^3 -> kg/m^3
#VAL3c[:,2] /= 1e1 #dyne/cm^2 -> Pascals(N/m^2)
#muTV=4.0/(3*0.74+1+VAL3c[:,6]/VAL3c[:,5])
#nzv=VAL3c[:,0].size
#MTW =np.loadtxt(filenames[1])[::-1,:]
#MTW[:,0] *= 1e3 #km -> m
#MTW[:,1] *= 1   #Kelvin
#MTW[:,2] /= 1e1 #dyne/cm^2 -> Pascals(N/m^2)
#nzm=MTW[:,0].size
#Combine both sets into single array
#hdata=np.zeros(nzv+nzm)
#hdata[0:nzv]=VAL3c[:,0]
#hdata[nzv:nzv+nzm]=MTW[:,0]
#hdata /= scales['length']
    
#pdata=np.zeros(nzv+nzm)
#pdata[0:nzv]=VAL3c[:,2]
#pdata[nzv:nzv+nzm]=MTW[:,2]
#pdata /= scales['energy density']
    
#Tdata=np.zeros(nzv+nzm)
#Tdata[0:nzv]=VAL3c[:,3]
#Tdata[nzv:nzv+nzm]=MTW[:,1] 
#Tdata /= scales['temperature']
    
#rdata=np.zeros(nzv+nzm)
#rdata[0:nzv]=VAL3c[:,1]
#rdata /= scales['density']   
#MTW[:,2], kB, mp and mu are in code units so no rescale needed
#rdata[nzv:nzv+nzm] = MTW[:,2]/physicalconstants['boltzmann']/MTW[:,1] *physicalconstants['proton_mass']*physicalconstants['mu']
    
#muofT = np.zeros(nzv+nzm) # mean molecular weight
#muofT[0:nzv] = muTV
#muofT[nzv:nzv+nzm] = physicalconstants['mu']
# interpolate total pressure, temperature and density profiles
#pdata_f = UnivariateSpline(hdata,np.log(pdata),k=1, s=0.25)
#Tdata_f = UnivariateSpline(hdata,np.log(Tdata),k=1, s=0.25)
#rdata_f = UnivariateSpline(hdata,np.log(rdata),k=1, s=0.25)
#s=0.0 to ensure all points are strictly used for ionisation state
#muofT_f = UnivariateSpline(hdata,np.log(muofT),k=1, s=0.0)
#pdata_i = np.exp(pdata_f(Z))
#Tdata_i = np.exp(Tdata_f(Z))
#rdata_i = np.exp(rdata_f(Z))
#muofT_i = np.exp(muofT_f(Z))

#source_data = [VAL3c,MTW] 

In [ ]:
print Tdata_f(Z)

## Recalculate density values

don't forget to adjust modelinfo values!

In [ ]:
for i3 in range(Nxyz[2]):
    for i2 in range(Nxyz[1]):
        for i1 in range(Nxyz[0]):
            alldat[i1,i2,i3,12]=1000*outdata['rho'][i1]
            alldat[i1,i2,i3,0]=1000*outdata['Z'][i1]

In [ ]:
print(alldat[48:80,64,64,0]/1.0e3)
print(alldat[48:80,64,64,12])

## write data to outputfile

In [ ]:
write_sac_ascii('test.ini',alldat,modelinfo)

## hydrostatic pressure balance